# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 16 2022 1:51PM,239779,10,0085775308,ISDIN Corporation,Released
1,Jun 16 2022 1:51PM,239779,10,0085775309,ISDIN Corporation,Released
2,Jun 16 2022 1:51PM,239779,10,0085775318,ISDIN Corporation,Released
3,Jun 16 2022 1:51PM,239779,10,0085775319,ISDIN Corporation,Released
4,Jun 16 2022 1:51PM,239779,10,0085775358,ISDIN Corporation,Released
5,Jun 16 2022 1:51PM,239779,10,0085775383,ISDIN Corporation,Released
6,Jun 16 2022 1:51PM,239779,10,0085775386,ISDIN Corporation,Released
7,Jun 16 2022 1:51PM,239779,10,0085775406,ISDIN Corporation,Released
8,Jun 16 2022 1:51PM,239779,10,0085775418,ISDIN Corporation,Released
9,Jun 16 2022 1:51PM,239779,10,0085775437,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,239772,Released,2
36,239775,Released,15
37,239777,Released,1
38,239778,Executing,1
39,239779,Released,14


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239772,NaN,NaN,2.0
239775,NaN,NaN,15.0
239777,NaN,NaN,1.0
239778,NaN,1.0,NaN
239779,NaN,NaN,14.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239672,0.0,0.0,29.0
239691,0.0,0.0,1.0
239692,0.0,0.0,1.0
239694,0.0,0.0,1.0
239696,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239672,0,0,29
239691,0,0,1
239692,0,0,1
239694,0,0,1
239696,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239672,0,0,29
1,239691,0,0,1
2,239692,0,0,1
3,239694,0,0,1
4,239696,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239672,,,29
1,239691,,,1
2,239692,,,1
3,239694,,,1
4,239696,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 16 2022 1:51PM,239779,10,ISDIN Corporation
14,Jun 16 2022 1:44PM,239778,12,"Uniderm USA, Inc"
15,Jun 16 2022 1:34PM,239777,10,Emerginnova
16,Jun 16 2022 1:26PM,239775,20,Alumier Labs Inc.
31,Jun 16 2022 12:54PM,239772,10,Emerginnova
33,Jun 16 2022 12:40PM,239770,10,ISDIN Corporation
67,Jun 16 2022 12:38PM,239769,10,Bio-PRF
69,Jun 16 2022 12:27PM,239767,15,"Virtus Pharmaceuticals, LLC"
82,Jun 16 2022 12:24PM,239765,15,"Person & Covey, Inc."
103,Jun 16 2022 11:58AM,239763,16,Sartorius Stedim North America


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 16 2022 1:51PM,239779,10,ISDIN Corporation,,,14
1,Jun 16 2022 1:44PM,239778,12,"Uniderm USA, Inc",,1,
2,Jun 16 2022 1:34PM,239777,10,Emerginnova,,,1
3,Jun 16 2022 1:26PM,239775,20,Alumier Labs Inc.,,,15
4,Jun 16 2022 12:54PM,239772,10,Emerginnova,,,2
5,Jun 16 2022 12:40PM,239770,10,ISDIN Corporation,,17,17
6,Jun 16 2022 12:38PM,239769,10,Bio-PRF,,1,1
7,Jun 16 2022 12:27PM,239767,15,"Virtus Pharmaceuticals, LLC",,,13
8,Jun 16 2022 12:24PM,239765,15,"Person & Covey, Inc.",2,14,5
9,Jun 16 2022 11:58AM,239763,16,Sartorius Stedim North America,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 1:51PM,239779,10,ISDIN Corporation,14,,
1,Jun 16 2022 1:44PM,239778,12,"Uniderm USA, Inc",,1,
2,Jun 16 2022 1:34PM,239777,10,Emerginnova,1,,
3,Jun 16 2022 1:26PM,239775,20,Alumier Labs Inc.,15,,
4,Jun 16 2022 12:54PM,239772,10,Emerginnova,2,,
5,Jun 16 2022 12:40PM,239770,10,ISDIN Corporation,17,17,
6,Jun 16 2022 12:38PM,239769,10,Bio-PRF,1,1,
7,Jun 16 2022 12:27PM,239767,15,"Virtus Pharmaceuticals, LLC",13,,
8,Jun 16 2022 12:24PM,239765,15,"Person & Covey, Inc.",5,14,2
9,Jun 16 2022 11:58AM,239763,16,Sartorius Stedim North America,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 1:51PM,239779,10,ISDIN Corporation,14,,
1,Jun 16 2022 1:44PM,239778,12,"Uniderm USA, Inc",,1,
2,Jun 16 2022 1:34PM,239777,10,Emerginnova,1,,
3,Jun 16 2022 1:26PM,239775,20,Alumier Labs Inc.,15,,
4,Jun 16 2022 12:54PM,239772,10,Emerginnova,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 1:51PM,239779,10,ISDIN Corporation,14.0,NaN,NaN
1,Jun 16 2022 1:44PM,239778,12,"Uniderm USA, Inc",NaN,1.0,NaN
2,Jun 16 2022 1:34PM,239777,10,Emerginnova,1.0,NaN,NaN
3,Jun 16 2022 1:26PM,239775,20,Alumier Labs Inc.,15.0,NaN,NaN
4,Jun 16 2022 12:54PM,239772,10,Emerginnova,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 1:51PM,239779,10,ISDIN Corporation,14.0,0.0,0.0
1,Jun 16 2022 1:44PM,239778,12,"Uniderm USA, Inc",0.0,1.0,0.0
2,Jun 16 2022 1:34PM,239777,10,Emerginnova,1.0,0.0,0.0
3,Jun 16 2022 1:26PM,239775,20,Alumier Labs Inc.,15.0,0.0,0.0
4,Jun 16 2022 12:54PM,239772,10,Emerginnova,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2397448,56.0,28.0,1.0
12,719273,1.0,3.0,0.0
15,719290,26.0,14.0,2.0
16,1198733,5.0,0.0,0.0
19,239752,1.0,0.0,0.0
20,1678043,59.0,18.0,4.0
22,479449,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2397448,56.0,28.0,1.0
1,12,719273,1.0,3.0,0.0
2,15,719290,26.0,14.0,2.0
3,16,1198733,5.0,0.0,0.0
4,19,239752,1.0,0.0,0.0
5,20,1678043,59.0,18.0,4.0
6,22,479449,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,56.0,28.0,1.0
1,12,1.0,3.0,0.0
2,15,26.0,14.0,2.0
3,16,5.0,0.0,0.0
4,19,1.0,0.0,0.0
5,20,59.0,18.0,4.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,56.0
1,12,Released,1.0
2,15,Released,26.0
3,16,Released,5.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,22
Status,,,,,,,
Completed,1.0,0.0,2.0,0.0,0.0,4.0,0.0
Executing,28.0,3.0,14.0,0.0,0.0,18.0,0.0
Released,56.0,1.0,26.0,5.0,1.0,59.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,22
0,Completed,1.0,0.0,2.0,0.0,0.0,4.0,0.0
1,Executing,28.0,3.0,14.0,0.0,0.0,18.0,0.0
2,Released,56.0,1.0,26.0,5.0,1.0,59.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,22
0,Completed,1.0,0.0,2.0,0.0,0.0,4.0,0.0
1,Executing,28.0,3.0,14.0,0.0,0.0,18.0,0.0
2,Released,56.0,1.0,26.0,5.0,1.0,59.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()